# Setup

In [1]:
# import your standard packages
%run ../../../global_tools/start.py

# import your local functions
sys.path.insert(1, '../../code/')
from local_functions import *

# make sure the figures plot inline rather than at the end
%matplotlib inline



Default libraries loaded.


# Paths and Parameters

In [2]:
outfn = '../../data/sla_to_awg.nc'

# Read Data

In [3]:
ds_SLA = xr.open_dataset('../../data/sla_processed.nc')
ds_SLA

<xarray.Dataset>
Dimensions:           (day_time: 10013, lat: 138, lon: 216, month: 12, time: 329)
Coordinates:
  * lon               (lon) float64 48.62 48.88 49.12 ... 101.9 102.1 102.4
  * lat               (lat) float64 -1.375 -1.125 -0.875 ... 32.38 32.62 32.88
  * time              (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2020-05-01
  * day_time          (day_time) datetime64[ns] 1993-01-01 ... 2020-05-31
  * month             (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    daily_sla         (day_time, lat, lon) float64 ...
    mon_sla           (time, lat, lon) float64 ...
    mon_sla_mon_clim  (month, lat, lon) float64 ...
    mon_sla_mon_anom  (time, lat, lon) float64 ...

In [4]:
ds_AWG = xr.open_dataset('../../data/awg_processed.nc')
ds_AWG

<xarray.Dataset>
Dimensions:  (pres: 201, time: 146472)
Coordinates:
    prof     (time) int32 ...
  * time     (time) datetime64[ns] 1958-06-01 1958-06-02 ... 2020-02-07
  * pres     (pres) int32 0 5 10 15 20 25 30 35 ... 970 975 980 985 990 995 1000
Data variables:
    temp     (time, pres) float32 ...
    psal     (time, pres) float32 ...
    doxy     (time, pres) float32 ...
    lat      (time) float32 ...
    lon      (time) float32 ...
    type     (time) int32 ...

# Regrid

In [6]:
# find unique time values of awg data
time_key = np.unique(ds_AWG.time)

# find all the days there are measurements
full_ind = np.full((ds_SLA.daily_sla.shape[0]),False)

for tt in range(time_key.shape[0]):
    ind = ds_SLA.daily_sla.day_time == time_key[tt]
    full_ind = full_ind + ind

# take only times where awg measurements were made
sla = np.array(ds_SLA.daily_sla[full_ind,:,:])
sla_time = np.array(ds_SLA.daily_sla.day_time[full_ind])

xx,yy = np.meshgrid(ds_SLA.lon,ds_SLA.lat)
xx = xx.flatten()
yy = yy.flatten()

lons= np.array(ds_AWG.lon)
lats = np.array(ds_AWG.lat)
times = np.array(ds_AWG.time)

daily_sla_AWG = np.zeros(len(times))*np.nan
lon_AWG = np.zeros(len(times))*np.nan
lat_AWG = np.zeros(len(times))*np.nan
time_AWG = np.array(ds_AWG.time)

for tt in tqdm(range(sla.shape[0])):
    
    sla_tmp = np.array(sla[tt,:,:])
    indq = times == sla_time[tt]
    lonq = lons[indq]
    latq = lats[indq]

    # find the lons and lats of TCD vals at this time

    points = np.array( (xx,yy) ).T
    values = sla_tmp.flatten()


    daily_sla_AWG[indq] = griddata(points, values, (lonq,latq) ,method='linear' )
    lon_AWG[indq] = lonq
    lat_AWG[indq] = latq



100%|██████████| 7106/7106 [49:37<00:00,  2.41it/s]


In [10]:
# convert to xarray dataset
ds=xr.Dataset(coords={'time': time_AWG})

# add variables to dataset

# AWG
ds["daily_sla"]=xr.DataArray(daily_sla_AWG,dims = ['time'],
                     coords =[time_AWG])
ds["lon"]=xr.DataArray(lon_AWG,dims = ['time'],
                     coords =[time_AWG])
ds["lat"]=xr.DataArray(lat_AWG,dims = ['time'],
                     coords =[time_AWG])

ds.to_netcdf(outfn,mode='w',format = "NETCDF4")

ds

<xarray.Dataset>
Dimensions:    (time: 146472)
Coordinates:
  * time       (time) datetime64[ns] 1958-06-01 1958-06-02 ... 2020-02-07
Data variables:
    daily_sla  (time) float64 nan nan nan nan ... -0.04217 0.05087 -0.1005
    lon        (time) float64 nan nan nan nan nan ... 86.88 64.74 53.53 84.37
    lat        (time) float64 nan nan nan nan nan ... 0.7947 14.39 11.25 1.808

In [30]:
daily_sla_AWG[~np.isnan(daily_sla_AWG)]

array([ 0.07522321,  0.05941689,  0.05987612, ..., -0.04433869,
       -0.06138302,  0.27052401])